In [1]:
%cd /home/maximilianheil/fungiclef-2024/

/home/maximilianheil/fungiclef-2024


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [27]:
%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
from pyspark.sql.types import StringType
import pyspark.sql.functions as f
import pyspark.ml as ml
from pyspark.ml.feature import Tokenizer
import fungiclef.embedding.transforms as trans
from fungiclef.utils import get_spark

In [4]:
spark = get_spark(**{
    "spark.sql.parquet.enableVectorizedReader": False, 
})
display(spark)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/21 15:12:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/21 15:12:21 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
24/04/21 15:12:23 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:

gcs_parquet_path = "gs://dsgt-clef-fungiclef-2024/dev/dev_train/"

#gcs_parquet_path = "gs://dsgt-clef-fungiclef-2024/data/parquet/"
#input_folder = f"DF20"



dev_df = spark.read.parquet(gcs_parquet_path)
dev_df.printSchema()
dev_df.count()

root
 |-- image_path: string (nullable = true)
 |-- data: binary (nullable = true)
 |-- observationID: long (nullable = true)
 |-- year: double (nullable = true)
 |-- month: double (nullable = true)
 |-- day: double (nullable = true)
 |-- countryCode: string (nullable = true)
 |-- locality: string (nullable = true)
 |-- taxonID: double (nullable = true)
 |-- scientificName: string (nullable = true)
 |-- kingdom: string (nullable = true)
 |-- phylum: string (nullable = true)
 |-- class: string (nullable = true)
 |-- order: string (nullable = true)
 |-- family: string (nullable = true)
 |-- genus: string (nullable = true)
 |-- specificEpithet: string (nullable = true)
 |-- taxonRank: string (nullable = true)
 |-- species: string (nullable = true)
 |-- level0Gid: string (nullable = true)
 |-- level0Name: string (nullable = true)
 |-- level1Gid: string (nullable = true)
 |-- level1Name: string (nullable = true)
 |-- level2Gid: string (nullable = true)
 |-- level2Name: string (nullable = tr

3686

# Transform number to txt

In [7]:
def cast_floats_to_int(df, list_of_cols):
    for col in list_of_cols:
        df = df.withColumn(col, dev_df[col].cast('int'))
    return df

numbers_to_mont_dict = {
    1 : "January",
    2 : "February",
    3 : "March",
    4 : "April",
    5 : "May",
    6 : "June",
    7 : "July",
    8 : "August",
    9 : "September",
    10 : "October",
    11 : "November",
    12 : "December"
}

def cast_int_to_nl_txt(mapping):
    def translate_(col):
        return mapping.get(col)
    return f.udf(translate_, StringType())

dev_df = cast_floats_to_int(dev_df, ['month'])
dev_df = dev_df.withColumn('month', cast_int_to_nl_txt(numbers_to_mont_dict)(dev_df['month']))

# Create Natural Text

In [9]:
ignore_cols = ['class', 'phylum', 'Latitude', 'Longitude', 'CoorUncert', 'txt_data', 'taxonRank', 'class_id', 'kingdom', 'image_path', 'data', 'observationID', 'year', 'ImageUniqueID', 'rightsHolder', 'day', 'countryCode', 'taxonID', 'level0Gid', 'level0Name', 'level1Gid', 'level2Gid']
relevant_columns = [col for col in  dev_df.columns if col not in ignore_cols]
relevant_columns

['month',
 'locality',
 'scientificName',
 'order',
 'family',
 'genus',
 'specificEpithet',
 'species',
 'level1Name',
 'level2Name',
 'Substrate',
 'Habitat',
 'MetaSubstrate',
 'poisonous',
 'filename']

In [10]:
# concat all txt data in one col txt_data

transformed_cols = [f.concat(f.lit(f"{col_name} "), f.col(col_name).cast("string")).alias(col_name) for col_name in relevant_columns]

dev_df = dev_df.withColumn("txt_data", f.concat_ws(", ", *transformed_cols))

dev_df.select("txt_data").show(1, vertical=True, truncate=False)

-RECORD 0------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 txt_data | month August, locality Tofte Skov, scientificName Russula ochroleuca (Pers.) Fr., order Russulales, family Russulaceae, genus Russula, specificEpithet ochroleuca, species Russula ochroleuca, level1Name Nordjylland, level2Name Mariagerfjord, Substrate soil, Habitat Mixed woodland (with coniferous and deciduous trees), MetaSubstrate jord, poisonous 0 
only showing top 1 row



# 

In [11]:
tokenizer = Tokenizer(outputCol="txt_token")
tokenizer.setInputCol("txt_data")

tokenized_df = tokenizer.transform(dev_df).select('txt_token', 'txt_data')

tokenized_df = tokenized_df.withColumn("len_txt_token", f.size(f.col("txt_token")))
tokenized_df = tokenized_df.withColumn("raw_len", f.length(f.col("txt_data")))

tokenized_df.sort('len_txt_token', ascending=False).show(10, vertical=False)

+--------------------+--------------------+-------------+-------+
|           txt_token|            txt_data|len_txt_token|raw_len|
+--------------------+--------------------+-------------+-------+
|[month, july,, lo...|month July, local...|           47|    411|
|[month, september...|month September, ...|           47|    420|
|[month, september...|month September, ...|           47|    389|
|[month, september...|month September, ...|           47|    416|
|[month, september...|month September, ...|           47|    416|
|[month, september...|month September, ...|           47|    416|
|[month, september...|month September, ...|           47|    389|
|[month, july,, lo...|month July, local...|           47|    411|
|[month, october,,...|month October, lo...|           47|    412|
|[month, october,,...|month October, lo...|           46|    416|
+--------------------+--------------------+-------------+-------+
only showing top 10 rows



In [47]:
# Init DINOv2 wrapper
clip = trans.WrappedCLIPV2(input_cols=["data","txt_data"], output_cols=['image', 'text', 'dot'])

# Create Pipeline
pipeline = ml.Pipeline(stages=[clip]) #, dctn

# Fit pipeline to DF
model = pipeline.fit(dev_df)

# Apply the model to transform the DF
transformed_df = model.transform(dev_df).cache()

TypeError: predict_batch_udf() missing 1 required keyword-only argument: 'return_type'

In [ ]:
transformed_df.select(["ImageUniqueID", "species", "Output.*"]).show(n=10)

Unused or unrecognized kwargs: padding, truncation.==>              (3 + 1) / 4]
Unused or unrecognized kwargs: padding, truncation.
Unused or unrecognized kwargs: padding, truncation.
Unused or unrecognized kwargs: padding, truncation.
Unused or unrecognized kwargs: padding, truncation.
Unused or unrecognized kwargs: padding, truncation.
Unused or unrecognized kwargs: padding, truncation.
Unused or unrecognized kwargs: padding, truncation.
Unused or unrecognized kwargs: padding, truncation.
Unused or unrecognized kwargs: padding, truncation.
Unused or unrecognized kwargs: padding, truncation.
Unused or unrecognized kwargs: padding, truncation.
Unused or unrecognized kwargs: padding, truncation.
Unused or unrecognized kwargs: padding, truncation.
Unused or unrecognized kwargs: padding, truncation.
Unused or unrecognized kwargs: padding, truncation.
Unused or unrecognized kwargs: padding, truncation.
Unused or unrecognized kwargs: padding, truncation.
Unused or unrecognized kwargs: padd

+-----------------+--------------------+--------------------+--------------------+--------------------+
|    ImageUniqueID|             species|               image|                text|                 dot|
+-----------------+--------------------+--------------------+--------------------+--------------------+
|2238067012-303186|  Russula ochroleuca|[0.017991642, 0.0...|[-0.010918789, 0....|[0.28513116, 0.30...|
|  2238175058-8780|    Amanita muscaria|[-0.002541536, 0....|[7.609477E-4, 0.0...|[0.2675045, 0.299...|
| 2238308944-85884|  Russula ochroleuca|[-0.003997852, 0....|[-0.023495277, 0....|[0.3154308, 0.311...|
|2238454170-165728|      Russula adusta|[0.022566577, 0.0...|[-0.01159004, -0....|[0.27796984, 0.26...|
| 2238500843-23035|       Imleria badia|[0.022290628, 0.0...|[-0.06541335, -0....|[0.29248983, 0.29...|
|2238513675-173334|  Lactarius blennius|[0.01703629, 0.05...|[-0.0039188997, 0...|[0.2854126, 0.257...|
| 2238527433-27340|  Russula ochroleuca|[0.018639402, 0.0...|[-0

In [41]:
transformed_df.select(["ImageUniqueID", "species", "Output.*"]).show(n=10)

Unused or unrecognized kwargs: padding, truncation.==>              (3 + 1) / 4]
Unused or unrecognized kwargs: padding, truncation.
Unused or unrecognized kwargs: padding, truncation.
Unused or unrecognized kwargs: padding, truncation.
Unused or unrecognized kwargs: padding, truncation.
Unused or unrecognized kwargs: padding, truncation.
Unused or unrecognized kwargs: padding, truncation.
Unused or unrecognized kwargs: padding, truncation.
Unused or unrecognized kwargs: padding, truncation.
Unused or unrecognized kwargs: padding, truncation.
Unused or unrecognized kwargs: padding, truncation.
Unused or unrecognized kwargs: padding, truncation.
Unused or unrecognized kwargs: padding, truncation.
Unused or unrecognized kwargs: padding, truncation.
Unused or unrecognized kwargs: padding, truncation.
Unused or unrecognized kwargs: padding, truncation.
Unused or unrecognized kwargs: padding, truncation.
Unused or unrecognized kwargs: padding, truncation.
Unused or unrecognized kwargs: padd

+-----------------+--------------------+--------------------+--------------------+--------------------+
|    ImageUniqueID|             species|               image|                text|                 dot|
+-----------------+--------------------+--------------------+--------------------+--------------------+
|2238067012-303186|  Russula ochroleuca|[0.017991642, 0.0...|[-0.010918789, 0....|[0.28513116, 0.30...|
|  2238175058-8780|    Amanita muscaria|[-0.002541536, 0....|[7.609477E-4, 0.0...|[0.2675045, 0.299...|
| 2238308944-85884|  Russula ochroleuca|[-0.003997852, 0....|[-0.023495277, 0....|[0.3154308, 0.311...|
|2238454170-165728|      Russula adusta|[0.022566577, 0.0...|[-0.01159004, -0....|[0.27796984, 0.26...|
| 2238500843-23035|       Imleria badia|[0.022290628, 0.0...|[-0.06541335, -0....|[0.29248983, 0.29...|
|2238513675-173334|  Lactarius blennius|[0.01703629, 0.05...|[-0.0039188997, 0...|[0.2854126, 0.257...|
| 2238527433-27340|  Russula ochroleuca|[0.018639402, 0.0...|[-0

In [88]:
transformed_df.select(["ImageUniqueID", "species", "transformed_data"]).show(n=10)

+-----------------+--------------------+--------------------+
|    ImageUniqueID|             species|    transformed_data|
+-----------------+--------------------+--------------------+
| 2237920504-77555|      Russula adusta|[0.009258113, 0.0...|
|  2237966683-5021|      Russula adusta|[0.019642368, 0.0...|
| 2237968009-79449|      Russula adusta|[0.0032701602, 0....|
| 2238149799-82085|Neoboletus luridi...|[-0.0048931465, 0...|
|  2238150033-7827|       Imleria badia|[-0.0015484869, 0...|
|2238171177-306291|  Lactarius blennius|[0.003171264, 0.0...|
| 2238209494-84510|Neoboletus luridi...|[0.022397494, 0.0...|
| 2238331048-11804|  Russula ochroleuca|[-0.014820853, 0....|
| 2238428423-15997|  Russula ochroleuca|[0.024092644, 0.0...|
| 2238455499-91533|  Lactarius blennius|[0.0208195, 0.047...|
+-----------------+--------------------+--------------------+
only showing top 10 rows



In [ ]:

# Init Descrite Cosine Transform wrapper
dctn = trans.DCTN(input_col="transformed_data", output_col="dctn_data")

# Create Pipeline
pipeline = ml.Pipeline(stages=[dino, dctn]) #, dctn

# Fit pipeline to DF
model = pipeline.fit(dev_subset_df)

# Apply the model to transform the DF
transformed_df = model.transform(dev_subset_df).cache()

# Show results
transformed_df.select(["ImageUniqueID", "species", "transformed_data"]).show(n=10)